In [7]:
'''
File : Notebook Vision
Author : Benoît Gallois
Date : 19 nov 2023
Definition of the vision functions that can detect the position of the obstacles, robot and goal area.
'''

import cv2
import numpy as np

'''
IMAGE_PATH = "map3.jpeg"

# Definition of the colours thresholds
LOWER_RED = np.array([0, 100, 100])
UPPER_RED = np.array([10, 255, 255])

LOWER_BLUE = np.array([100, 100, 100])
UPPER_BLUE = np.array([140, 255, 255])

LOWER_GREEN = np.array([40, 40, 40])
UPPER_GREEN = np.array([80, 255, 255])

LOWER_BLACK = np.array([0, 0, 0])
UPPER_BLACK = np.array([179, 255, 30])

# Definition of the size of contours considered as noise
NOISY_CONTOUR_LENGHT = 2000
'''


def detect_area(image_path, lower_colour, upper_colour):
    '''
    @brief   Detects areas corresponding to a color and returns the coordinates of the vertices of these areas.
 
    @param   image_path   -> "folder_of_image/name_image.jpeg"
             lower_colour -> LOWER_RED, LOWER_BLACK, LOWER_BLUE, LOWER_GREEN
             upper_colour -> UPPER_RED, UPPER_BLACK, UPPER_BLUE, UPPER_GREEN
    
    @return  coords       -> list of the coordinates of the vertices for each areas
    '''
    
    image = cv2.imread(image_path)
    height, width, _ = image.shape      # Give the size of the image

    # Converts the image in the HSV space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Filter the image to retain only pixels of the desired color
    mask = cv2.inRange(hsv, lower_colour, upper_colour)

    # Blur masks to reduce noise
    blurred_mask = cv2.GaussianBlur(mask, (5, 5), 0)

    # Find contours in the filtered mask
    contours, _ = cv2.findContours(blurred_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # List to store coordinates of detected zones
    coords = []

    # Browse all contours found
    for contour in contours:
        
        # Ignore small contours that could be noise
        if cv2.contourArea(contour) > NOISY_CONTOUR_LENGHT:
            
            # Get the coordinates of the rectangle enclosing the area
            x, y, w, h = cv2.boundingRect(contour)

            # Add the coordinates of the zone's vertices with a change of reference point (bottom left corner)
            coords.append([(x, height - y), (x + w, height - y), (x + w, height - y - h), (x, height - y - h)])

    return coords





def calculate_robot_direction(front_area, back_area):
    '''
    @brief   Calculates the direction vector of the robot.
 
    @param   front_area   -> List of coordinates for the front area (blue)
             back_area    -> List of coordinates for the back area (green)
    
    @return  direction    -> tuple represents the direction vector coordinates
                             list represents the midpoint coordinates
    '''
    
    # Calculate the centroid of the front area (blue)
    front_centroid = np.mean(np.array(front_area).reshape(-1, 2), axis=0)
    
    # Calculate the centroid of the back area (green)
    back_centroid = np.mean(np.array(back_area).reshape(-1, 2), axis=0)
    
    # Calculate the direction vector of the robot
    direction = (front_centroid - back_centroid).astype(int)
    
    # Calculate the midpoint between the centroids
    midpoint = ((front_centroid + back_centroid) / 2).astype(int)
    
    return tuple(direction), midpoint





def calculate_area_centroid(area_coordinates):
    '''
    @brief   Calculates the centroid coordinates of an area.
 
    @param   area_coordinates -> List of coordinates area
    
    @return  centroid         -> List that represents the centroid coordinates
    '''
    
    # Calculate the centroid of the area
    centroid = np.mean(np.array(area_coordinates).reshape(-1, 2), axis=0)
    
    return centroid

In [6]:
# Find the contours of each areas
obstacles = detect_area(IMAGE_PATH, LOWER_BLACK, UPPER_BLACK)
goal_area = detect_area(IMAGE_PATH, LOWER_RED, UPPER_RED)
front_robot_area = detect_area(IMAGE_PATH, LOWER_BLUE, UPPER_BLUE)
back_robot_area = detect_area(IMAGE_PATH, LOWER_GREEN, UPPER_GREEN)

# Find the robot direction vector and its midpoint
robot_direction = calculate_robot_direction(front_robot_area[0], back_robot_area[0])

# Find the centroid of the goal area
goal_centroid = calculate_area_centroid(goal_area)

In [3]:
print(f"Coordonnées obstacles noirs :", obstacles)

print("Coordonnées zone d'arrivée :", goal_area)

print("Coordonnées zone bleue :", front_robot_area)

print("Coordonnées zone verte :", back_robot_area)

print("Vecteur direction du robot :", robot_direction[0])

print("Midpoint entre les centroids des zones verte et bleue :", robot_direction[1])

print("Centroid de la goal area :", goal_centroid)

Coordonnées obstacles noirs : [[(371, 1178), (1245, 1178), (1245, 906), (371, 906)], [(1605, 3481), (2100, 3481), (2100, 3203), (1605, 3203)]]
Coordonnées zone d'arrivée : [[(350, 2803), (1332, 2803), (1332, 1822), (350, 1822)]]
Coordonnées zone bleue : [[(1427, 2139), (2457, 2139), (2457, 1060), (1427, 1060)]]
Coordonnées zone verte : [[(1670, 1912), (2900, 1912), (2900, 659), (1670, 659)]]
Vecteur direction du robot : (-343, 314)
Midpoint entre les centroids des zones verte et bleue : [2113 1442]
Centroid de la goal area : [ 841.  2312.5]


## A Faire

- augmenter largeur des obstacles virtuellement 
- donner centroide de la zone verte -> centre entre les deux roues